In [1]:
import numpy as np
import pandas as pd
import string
from datasets import load_dataset
import nltk

In [2]:
def preprocess_dataset(dataset):
    # Convert from pyarrow to pandas df
    df = dataset.to_pandas()
    
    # Map letter answers to options list indices
    df['answer'] = df['answer'].map({'A':0, 'B':1, 'C':2, 'D':3})
    options_series = df['options']

    def is_valid_options(options_list):
        # Strip option, remove punctuation, return True if not all options are just numbers.
        is_valid = [not sent.strip().translate(str.maketrans('', '', string.punctuation)).isnumeric() for sent in options_list]
        return any(is_valid)

    # Map is_valid_options to every question
    is_valid_series = options_series.map(is_valid_options)
    num_dropped = is_valid_series.value_counts()[False]
    print(f'Dropped {num_dropped} rows with only numeric answers')
    return df[is_valid_series]


In [94]:
pd.set_option('display.max_colwidth', None)

def filt_df(df):
    
    ## Exclude phrase completion questions
    filt_df = df[ ~df['question'].str.contains('_')]
    print(filt_df.shape)

    ## Exclude "According to the passage" questions
    filt_df = filt_df[ ~filt_df['question'].str.contains('According to the passage')]
    print(filt_df.shape)

    ## Exclude questions shorter than 5 words
    filt_df = filt_df[ filt_df.question.str.replace(',','').str.split().str.len() > 5 ]
    print(filt_df.shape)
    
    ## Exclude contexts w/ less than n questions
    vc = filt_df['example_id'].value_counts().to_frame()
    vc.columns = ['Count']
    vc = vc[vc.Count > 0]
    filt_df = filt_df[filt_df['example_id'].isin(vc.index)]
    print(filt_df.shape)
    
    ## Exclude questions with answers shorter than 3 words
    filt_df['answer_text'] = filt_df.apply(lambda x: x['options'][x['answer']], axis=1)
    filt_df['answer_text_len'] = filt_df['answer_text'].str.replace(',','').str.split().str.len()
    filt_df = filt_df[ filt_df.answer_text_len >= 3 ]
    filt_df = filt_df.drop(['answer_text', 'answer_text_len'], axis=1)
    print(filt_df.shape)

    return filt_df

In [95]:
#pd.set_option('display.max_colwidth', None)

race = load_dataset('race', 'all')
race_train = race['train'].to_pandas()
#race_train

Reusing dataset race (/Users/Hasan/.cache/huggingface/datasets/race/all/0.1.0/5839ff74a429622f5f20cca69c5fcf0e87ac6d5fd2777c42b948000684829f7b)


In [96]:
#valid_df = filt_df(preprocess_dataset(race['validation']))
train_df = filt_df(preprocess_dataset(race['train']))
train_df

Dropped 1304 rows with only numeric answers
(41197, 5)
(40312, 5)
(38187, 5)
(38187, 5)
(31941, 5)


,example_id,article,answer,question,options
3,high19088.txt,"Last week I talked with some of my students about what they wanted to do after they graduated, and what kind of job prospects they thought they had.\nGiven that I teach students who are training to be doctors, I was surprised do find that most thought that they would not be able to get the jobs they wanted without ""outside help"". ""What kind of help is that?"" I asked, expecting them to tell me that they would need a or family friend to help them out.\n""Surgery ,"" one replied.\nI was pretty alarmed by that response. It seems that the graduates of today are increasingly willing to go under the knife to get ahead of others when it comes to getting a job .\nOne girl told me that she was considering surgery to increase her height. ""They break your legs, put in special extending screws, and slowly expand the gap between the two ends of the bone as it re-grows, you can get at least 5 cm taller!""\nAt that point, I was shocked. I am short, I can't deny that, but I don't think I would put myself through months of agony just to be a few centimetres taller. I don't even bother to wear shoes with thick soles, as I'm not trying to hide the fact that I am just not tall!\nIt seems to me that there is a trend towards wanting ""perfection"" , and that is an ideal that just does not exist in reality.\nNo one is born perfect, yet magazines, TV shows and movies present images of thin, tall, beautiful people as being the norm. Advertisements for slimming aids, beauty treatments and cosmetic surgery clinics fill the pages of newspapers, further creating an idea that ""perfection"" is a requirement, and that it must be purchased, no matter what the cost. In my opinion, skills, rather than appearance, should determine how successful a person is in his/her chosen career.",1,Which' s the best title for the passage?.,"[Young Graduates Have Higher Expectations, Young Graduates Look to Surgery for Better Jobs, Young Graduates' Opinion About Cosmetic Surgery, Young Graduates Face a Different Situation in Job-hunting]"
4,high15596.txt,"YUZHOU, HENAN -An accident in a central China coal mine killed 21 miners Saturday and left another 16 trapped underground , the government said.\nThe death\nrose to 26 Sunday morning as rescuers were battling to reach the 11 miners who were still trapped underground, rescue headquarters said.\nRescuers were battling to reach the 11 miners still trapped underground, but chances for them to survive were very slim, said Du Bo, deputy chief of the rescue headquarters.\n""Based upon past experience, the remaining 11 miners could be buried in coal dust, so the survival chances are frail,"" Du said.\nMore than 2,500 tons of coal dust smothered the pit after the gas leak , which hampered the rescue, said Du.\nThe gas outburst happened at 6:03 a.m. Saturday when 276 miners were working underground in the mine in Yuzhou City. A total of 239 workers escaped but 21 were found dead and 16 trapped.\nAn initial investigation showed that 173,500 cubic meters of gas leaked out in the accident. Liu Wenbin, a deputy chief engineer of the company that owns the mine, was in the pit when the accident happened. He organized the escape.\n""At around 6 a.m., I felt there was something wrong with the airflow in the shaft, and one of the team captains told me he also felt it and had already reported the problem,"" said Liu\nThe mine is owned by Pingyu Coal & Electric Co. Ltd., a company jointly established by four investors, including Zhong Ping Energy Chemical Group and China Power Investment Corp.",1,What could be the best title for this passage?,"[Death Toll Rises in an Accident in China, A Coal Mine Accident in Central China, An Accident in Central China, Coal Mine Accidents in China]"
6,high15596.txt,"YUZHOU, HENAN -An accident in a central China coal mine killed 21 miners Saturday and left another 16 trapped underground , the government said.\nThe death\nrose to 26 Sunday morning 

In [83]:
train_df.almost.value_counts()

7      3892
6      3802
8      3675
1      3521
5      3518
3      3417
9      3018
4      2983
2      2724
10     2440
11     1752
12     1215
13      857
14      513
15      333
16      205
17      111
18       72
19       49
20       37
22       13
21       11
23       11
24        7
26        3
25        3
105       1
34        1
29        1
27        1
Name: almost, dtype: int64

In [16]:
race = load_dataset('race', 'all')
race_train = race['train'].to_pandas()
print(race_train.shape)
vc = race_train['example_id'].value_counts().to_frame()
vc.columns = ['Count']
vc = vc[vc.Count > 5]
race_train = race_train[race_train['example_id'].isin(vc.index)]
print(race_train.shape)

Reusing dataset race (/Users/Hasan/.cache/huggingface/datasets/race/all/0.1.0/5839ff74a429622f5f20cca69c5fcf0e87ac6d5fd2777c42b948000684829f7b)



(87866, 5)
(217, 5)


In [15]:
vc

,Count
middle7941.txt,7
high4205.txt,6
high19876.txt,6
high11655.txt,6
high6122.txt,6
...,...
high17887.txt,1
high38.txt,1
high24041.txt,1
high12657.txt,1
